# Testing different SA methods 2/5
## nltk's sentiment analyzer
This code was partly taken from [here](www.nltk.org/howto/sentiment.html) TODO also attribute in thesis

In [7]:
# Imports
import nltk
import csv
import pickle
import random
import time

from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from gensim import corpora

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer(remove_stopwords=False)

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(tokenize(preprocess(text)), sentiment) for (topic, sentiment, id, text) in iterator]
    # The tweets are ordered, so shuffle for the training/test split
    random.shuffle(tweets)
    
# Split between training and test set
# 90% training, 10% test
threshold = int(0.9 * len(tweets))
train_tweets = tweets[:threshold]
test_tweets = tweets[threshold:]

In [8]:
sentim_analyzer = SentimentAnalyzer()
# Words with negation
all_words_neg = sentim_analyzer.all_words([nltk.sentiment.util.mark_negation(doc) for doc in tweets])
# WOrds without negation
all_words = sentim_analyzer.all_words(tweets)

We use simple unigram word features, handling negation if we want:

In [9]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=1) # Same minimum frequency used the gensim-dictionary
len(unigram_feats)

2709

In [10]:
sentim_analyzer.add_feat_extractor(nltk.sentiment.util.extract_unigram_feats, unigrams=unigram_feats)

We apply features to obtain a feature-value representation of our datasets:

In [11]:
training_set = sentim_analyzer.apply_features(train_tweets)
test_set = sentim_analyzer.apply_features(test_tweets)

We can now train our classifier on the training set, and subsequently output the evaluation results:

In [12]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [13]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.7525773195876289
F-measure [negative]: 0.6306306306306306
F-measure [neutral]: 0.8522167487684729
F-measure [positive]: 0.3384615384615385
Precision [negative]: 0.5384615384615384
Precision [neutral]: 0.8693467336683417
Precision [positive]: 0.4074074074074074
Recall [negative]: 0.7608695652173914
Recall [neutral]: 0.8357487922705314
Recall [positive]: 0.2894736842105263


In [41]:
# If its a good classifier, we can save it by running this cell
save_classifier = open("./../../models/naive_bayes/nltk_sentiment_analyzer_all_labels.pickle","wb")
pickle.dump(sentim_analyzer, save_classifier)
save_classifier.close()